In [54]:
import os
from pdfminer.high_level import extract_text

def extract_text_from_pdfs(root_folder):
    # Iterate over all subdirectories in the root folder
    for subdir, dirs, files in os.walk(root_folder):
        pdf_files = [f for f in files if f.endswith('.pdf')]
        
        # Iterate over all PDF files in the subdirectory
        for i, pdf_file in enumerate(pdf_files, start=1):
            pdf_path = os.path.join(subdir, pdf_file)
            
            # Extract text from the PDF file
            text = extract_text(pdf_path)
            
            # Save the text to a new .txt file in the same subdirectory
            txt_file = os.path.join(subdir, f'{i}.txt')
            with open(txt_file, 'w') as f:
                f.write(text)

# Call the function with the path to your root folder
extract_text_from_pdfs('C:\\Users\\Dell\\Desktop\\NLP Assignment 3\\')

In [59]:
import os
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from pdfminer.high_level import extract_text
vectorizer = CountVectorizer(input='filename')

def create_document_term_matrix_with_labels(root_folder):
        
    # Collect all .txt files in all subdirectories
    txt_files = []
    labels = []
    for subdir, dirs, files in os.walk(root_folder):
        for f in files:
            
            if f.endswith('.txt'):
                txt_files.append(os.path.join(subdir, f))
                labels.append(os.path.basename(subdir))  # Use the subfolder name as the label
    
    # Create a document-term matrix
    dtm = vectorizer.fit_transform(txt_files)
    
    # Convert the matrix to a pandas DataFrame
    df = pd.DataFrame(dtm.toarray(), columns=vectorizer.get_feature_names_out(), index=['Doc'+str(i+1) for i in range(dtm.shape[0])])
    
    # Add the labels to the DataFrame
    df['label'] = labels
    
    return df
df = create_document_term_matrix_with_labels('C:\\Users\\Dell\\Desktop\\NLP Assignment 3\\')
df_log = np.log1p(df.drop('label', axis=1))

In [60]:
df_log.shape

(50, 27983)

In [61]:
print(df_log.columns)

Index(['00', '000', '0000', '0000446779', '00006', '0001', '00010', '00011',
       '00013', '00018',
       ...
       '𝜇𝑦', '𝜇𝜌', '𝜌2', '𝜌th', '𝜌total', '𝜎2', '𝜎𝜃', '𝜓2', '𝜓l', '𝜓𝑙'],
      dtype='object', length=27983)


In [62]:
df['label']

Doc1     Asp
Doc2     Asp
Doc3     Asp
Doc4     Asp
Doc5     Asp
Doc6     Asp
Doc7     Asp
Doc8     Asp
Doc9     Asp
Doc10    Asp
Doc11     DL
Doc12     DL
Doc13     DL
Doc14     DL
Doc15     DL
Doc16     DL
Doc17     DL
Doc18     DL
Doc19     DL
Doc20     DL
Doc21    Med
Doc22    Med
Doc23    Med
Doc24    Med
Doc25    Med
Doc26    Med
Doc27    Med
Doc28    Med
Doc29    Med
Doc30    Med
Doc31     ML
Doc32     ML
Doc33     ML
Doc34     ML
Doc35     ML
Doc36     ML
Doc37     ML
Doc38     ML
Doc39     ML
Doc40     ML
Doc41    NLP
Doc42    NLP
Doc43    NLP
Doc44    NLP
Doc45    NLP
Doc46    NLP
Doc47    NLP
Doc48    NLP
Doc49    NLP
Doc50    NLP
Name: label, dtype: object

In [63]:
def pdf_to_txt_converter(doc_path):
    text = extract_text(doc_path)
    txt_path = os.path.splitext(doc_path)[0] + '.txt'
    # Save the text to a new .txt file in the same subdirectory
    with open(txt_path, 'w') as f:
        f.write(text)
    return None

def compute_cosine_similarity(new_doc):
    if new_doc.endswith('.pdf'):
        pdf_to_txt_converter(new_doc)
        new_doc = os.path.splitext(new_doc)[0] + '.txt'
    from sklearn.metrics.pairwise import cosine_similarity
    new_doc_vector = vectorizer.transform([new_doc])
    new_doc_row = pd.Series(new_doc_vector.toarray().flatten(), index=vectorizer.get_feature_names_out())
    new_doc_list = new_doc_row.drop('label').tolist()
    new_doc_list = np.log1p(new_doc_list)
    similarities = []
    for i in range(df_log.shape[0]):
        row = df_log.loc['Doc'+str(i+1)].tolist()
        sim = cosine_similarity(np.array(new_doc_list).reshape(1, -1), np.array(row).reshape(1, -1))
        similarities.append(sim[0][0])
    return similarities
#pdf_to_txt_converter('C:\\Users\\Dell\\Desktop\\NLP Assignment 3\\2312.16724.pdf')
x = compute_cosine_similarity(('C:\\Users\\Dell\\Desktop\\NLP Assignment 3\\2312.16724.pdf'))

In [64]:
len(x)

50

In [65]:
sublists = [x[i:i+10] for i in range(0, len(x), 10)]
averages = [np.mean(sublist) for sublist in sublists]
labels = ['Asp','DL', 'Med', 'ML', 'NLP']
label = labels[np.argmax(averages)]
print(label)

DL
